# Equations

## Necessary imports

In [1]:
import math
import random
import pandas as pd
import numpy as np

## Constants

In [2]:
####### Définitions des variables
ELRbase0 = 1600.
SUPthreshold = 0.2
CELLShive = 250000
x1, x2, x3, x4, x5 = 385, 30, 36, 155, 30
LIFESPANegg = 3
LIFESPANlarva = 5
LIFESPANpupa = 12
MORTALITYeggs = 0.03
MORTALITYlarvae = 0.01
MORTALITYpupae = 0.001
CANNIBALISMhungerbase = [0.23, 0.3, 0.58, 0.06, 0] # for i in [1,2,...,5]
MORTALITYbase = 0.01
MORTALITYnursing = 0.005
MORTALITYprocessing = 0.005
MORTALITYforaging = 0.035
LOADpollenforager = 0.06
LOADnectarforager = 0.04
TURNSnectarforager = 15
TURNSpollenforager = 10
FACTORforagingsuccess = 0.8
FACTORminpollenforagers = 0.01
FACTORforagingmax = 0.33
ProcessorsPerCell = 2
FACTORpollenstorage = 6
FACTORpollensavingmax = 0.3
RATIOnectar_to_honey = 0.4 # 20/50
w_nectar = 0.43
w_pollen = 0.23
w_cellsbase = 0.037
w_honey = 0.5
w_egg = 0.0001
w_pupa = 0.16
w_adult = 0.1
w_larva = [0.0002,0.00059, 0.00331, 0.0644, 0.160] # for i in [1,...,5]
w_hivebase = 14000 #("14,000g")
ERLstochrange = .05

#######Creations des tableaux
STORESpollenTab = np.zeros(366)
STOREShoneyTab = np.zeros(366)
STORESnectarTab = np.zeros(366)
BroodCellsTab = np.zeros(366)
BEESadultTab = np.zeros(366)
WEIGHTcolonyTab = np.zeros(366)
t = np.arange(366)
opt = 8 # equation choice (7 or 8) for CELLSeggs(t)
opt2 = 12 # equation choice (12 or 13) for CELLSlarvae(t)
opt3 = 15 # equation choice (15 or 16) for CELLSpupae(t)


## Initialisations

In [ ]:
####### Initialisation des valeurs à 0
STORESpollenTab[0] = 0
STOREShoneyTab[0] = 50000
STORESnectarTab[0] = 0
BroodCellsTab[0] = 0
BEESadultTab[0] = 15000
WEIGHTcolonyTab[0] = 50

####### Creation du dataframe
ar = np.array([STORESpollenTab,STOREShoneyTab,STORESnectarTab,BroodCellsTab,BEESadultTab,WEIGHTcolonyTab])
dFResult= pd.DataFrame({'STORESpollen' : STORESpollenTab, 'STOREShoney' : STOREShoneyTab, 'STORESnectar' : STORESnectarTab, 'BroodCells' : BroodCellsTab,'BEESadult' : BEESadultTab,'WEIGHTcolony' : WEIGHTcolonyTab})

####### Parameter swarming day
swd = 140

####### Test
NECTARNEEDlarva = np.arange(1,6,1.0)
POLLENNEEDlarva = np.arange(1,6,1.0)

####### Définitions des variables "inconnues"
POLLENNEEDadult = 1
POLLENNEEDnurse = 1
NECTARNEEDactiveforager = 1
NECTARNEEDadult = 1
NECTARNEEDnurse = 1

### Array Initialisations

In [ ]:
global CELLSemptyArray, SUPcombArray, ELRArray, CELLSeggsArray, CELLSlarvaeArray, CELLSpupaeArray, CELLSbroodArray, MORTALITYadultArray, BEESadultArray, INDEXflightArray, INDEXnectaroutsideArray, INDEXpollenoutsideArray

CELLSemptyArray = []
SUPcombArray = []
ELRArray = []
CELLSeggsArray = []
CELLSlarvaeArray = []
CELLSpupaeArray = []
CELLSbroodArray = []
MORTALITYadultArray = []
BEESadultArray = []
INDEXflightArray = []
INDEXnectaroutsideArray = []
INDEXpollenoutsideArray = []

## 2.1.   Modeling the queen’s egg laying behavior

In [ ]:
def season(t):
    # (1)
    return max(1-(1/(1+x1*math.exp(-2*t/x2))),1/(1+x3*math.exp(-2*(t-x4)/x5)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

graph = []

for i in range(360):
    graph.append(season(i))

plt.figure(figsize=(10,5))
plt.plot(np.linspace(1,360, 360), graph)

In [ ]:
def CELLSempty(t):
    # (2)
    if t>len(CELLSemptyArray):
        CELLSempty(t-1)
    if t==len(CELLSemptyArray):
        res = CELLShive(t) - CELLSbrood(t) - STORESpollen(t) - STORESnectar(t) - STOREShoney(t)
        CELLSemptyArray = np.append(CELLSemptyArray, res)
    return CELLSemptyArray[t]

In [ ]:
def SUPcomb(t):
    # (3)
    if t>len(SUPcombArray):
        SUPcomb(t-1)
    if t == len(SUPcombArray):
        if CELLSempty(t)/(CELLShive(t) + 1) < SUPthreshold:
            res = CELLSempty(t)/((CELLShive(t) + 1) * SUPthreshold)
        else:
            res = 1
        SUPcombArray = np.append(SUPcombArray, res)
    return SUPcombArray[t]

In [ ]:
def ELRstoch(t):
    # (4)
    return random.uniform(-ERLstochrange,ERLstochrange)

In [ ]:
def ELR(t):
    # (5)
    global ELRArray
    if t > len(ELRArray):
        res = ELRbase(t)
    elif t == len(ELRArray):
        res = ELRbase(t) * (1 + ELRstoch(t)) * (1 - season(t)) * SUPcomb(t)
        ELRArray = np.append(ELRArray, res)
    return ELRArray[t]

## 2.2. Modeling the immature stages

### 2.2.1 Eggs

In [ ]:
#VOIR INIT
def EGGS(i,t):
    # (6)
    if i == 1:
        res = ELR(t-1) * (1 - MORTALITYeggs)
    elif i in [2,3]:
        res = EGGS(i-1,t-1) * (1 - MORTALITYeggs)
    return res

In [ ]:
def CELLSeggs(t):
    # (7)
    if t > len(CELLSeggsArray):
        CELLSeggs(t-1)
    elif t == len(CELLSeggsArray):
        sum = 0
        for i in range(1,LIFESPANegg):
            sum += EGGS(i,t)
        CELLSeggsArray = np.append(CELLSeggsArray, sum)
    return CELLSeggsArray[t]

### 2.2.2. Larvae (unsealed)

In [ ]:
def SURVIVALlarvae(i,t):
    # (9)
    return (1 - CANNIBALISMlarvae(i,t)) * (1 - MORTALITYlarvae)

In [ ]:
def CANNIBALISMlarvae(i,t):
    # (10)
    return CANNIBALISMhungerbase[i] * (1 - (INDEXpollensituation(t-1) * INDEXnursingquality(t-1)))

In [ ]:
#VOIR INIT
def LARVAE(i,t):
    # (11)
    if i == 1:
        res = EGGS(LIFESPANegg,t-1)*SURVIVALlarvae(1,t)
    elif 1 < i and i <= LIFESPANlarva:
        res = LARVAE(i-1,t-1) * SURVIVALlarvae(i,t)
    return res

In [ ]:
def CELLSlarvae(t):
    # (12)
    if t > len(CELLSlarvaeArray):
        CELLSlarvae(t-1)
    elif t == len(CELLSlarvaeArray):
        sum = 0
        for i in range(1, LIFESPANlarva):
            sum += LARVAE(i,t)
        CELLSlarvaeArray = np.append(CELLSlarvaeArray, sum)
    return CELLSlarvaeArray[t] 

In [ ]:
# def CELLSlarvae13(t):
#     # (13)
#     # alternative to (12)
    
#     def product(i):
#         prdct = 1
#         for k in range(i+1):
#             if k != 0:
#                 prdct *= SURVIVALlarvae(k,t-i+k)
#         return prdct
    
#     sum = 0
#     for i in range(LIFESPANlarva+1):
#         if i != 0:
#             sum += (ELR(t-i-LIFESPANegg)*math.pow(1-MORTALITYeggs,i)*product(i))  
    
#     return sum

In [ ]:
# def CELLSlarvae(t,opt):
#     # (12) / (13)
#     if opt == 12:
#         return CELLSlarvae12(t)
#     elif opt == 13:
#         return CELLSlarvae13(t)
#     else:
#         raise Exception("opt must be 12 or 13")

In [ ]:
#VOIR INIT
def PUPAE(i,t):
    # (14)
    if i == 1:
        res = LARVAE(LIFESPANlarva,t-1) * (1 - MORTALITYpupae)
    elif 1 < i and i <= LIFESPANpupa:
        res = PUPAE(i-1,t-1) * (1 - MORTALITYpupae)
    return res

In [ ]:
def CELLSpupae(t):
    # (15)
    if t > len(CELLSpupaeArray):
        CELLSpupae(t-1)
    elif t == len(CELLSpupaeArray):
        sum = 0
        for i in range(1,LIFESPANpupa):
            sum += PUPAE((i,t))
        CELLSpupaeArray = np.append(CELLSpupaeArray, sum)
    return CELLSpupaeArray[t] 

In [ ]:
# def CELLSpupae16(t):
#     # (16)
#     # alternative to (15)
    
#     def product(i):
#         prdct = 1
#         for k in range(LIFESPANlarva+1):
#             if k != 0:
#                 prdct *= SURVIVALlarvae(k,t-i-LIFESPANlarva+k)
#         return prdct
    
#     sum = 0
#     for i in range(LIFESPANpupa+1):
#         if i != 0:
#             sum += (ELR(t-i-LIFESPANegg-LIFESPANlarva)*math.pow(1-MORTALITYeggs,LIFESPANegg)*product(i)*math.pow(1-MORTALITYpupae,i))  
    
#     return sum

In [ ]:
# def CELLSpupae(t,opt):
#     # (15) / (16)
#     if opt == 15:
#         return CELLSpupae15(t)
#     elif opt == 16:
#         return CELLSpupae16(t)
#     else:
#         raise Exception("opt must be 15 or 16")

In [ ]:
def CELLSbrood(t):
    # (17)
    if t > len(CELLSbroodArray):
        CELLSbrood(t-1)
    elif t == len(CELLSbroodArray):
        res = CELLSeggs(t) + CELLSlarvae(t) + CELLSpupae(t)
        CELLSbroodArray = np.append(CELLSbroodArray, res)
    return CELLSbroodArray[t]

## 2.3 Modeling the population of adult bees

In [ ]:
def MORTALITYadult(t) :
    # (18)
    if t > len(MORTALITYadultArray):
        MORTALITYadult(t-1)
    elif t == len(MORTALITYadultArray):
        res = MORTALITYadultbase
        res += (MORTALITYnursing * NURSES(t - 1) / BEESadult(t - 1) + 1)
        res += (MORTALITYprocessing * PROCESSORS(t - 1) / BEESadult(t - 1) + 1)
        res += (MORTALITYforaging * FORAGERSactive(t - 1) / BEESadult(t - 1) + 1)
        MORTALITYadultArray = np.append(MORTALITYadultArray, res)
    return MORTALITYadultArray[t]

In [ ]:
def BEESadult(t) : 
    # (19)
    if t>len(BEESadultArray):
        BEESadult(t-1)
    elif t==len(BEESadultArray):
        res = (BEESadult(t-1) + PUPAE(LIFESPANpupa, t-1))*(1 - MORTALITYadult(t) - BEESswarm(t))
        BEESadultArray = np.append(BEESadultArray, res)
    return BEESadultArray[t]

## 2.4 Modeling the influence of the environment

In [ ]:
#VOIR ICI
def INDEXrain(t) :
    # (20)
	return (1 - RAIN(t))

In [ ]:
#VOIR ICI
def RAIN(t) :
    # (21)
	res = HOURSraining_during_daylight(t) / HOURSdaylight(t)
	return res

In [ ]:
#VOIR ICI
def INDEXtemperature(t) :
    # (22)
	if ((TEMP(t) <= 14) or (TEMP(t) > 40)) :
		res = 0
	elif (TEMP(t) <= 22) :
		res = (TEMP(t) - 14) / 8
	elif (TEMP(t) <= 32) : 
		res = 1
	else : 
		res = (40 - TEMP(t)) / 8 

	return res

In [ ]:
def INDEXflight(t): 
    # (23)
    if t > len(INDEXflightArray):
        INDEXflight(t-1)
    elif t == len(INDEXflightArray):
        res = (INDEXrain(t) * INDEXtemperature(t))
        INDEXflightArray = np.append(INDEXflightArray, res)
    return INDEXflightArray[t]

In [ ]:
def INDEXnectaroutside(t) :
    # (24)
    if t > len(INDEXnectaroutsideArray):
        INDEXnectaroutside(t-1)
    elif t == len(INDEXnectaroutsideArray):
        res = min((1 - season(t)) * 1.5, 1)
        INDEXnectaroutsideArray = np.appen(INDEXnectaroutsideArray, res)
    return INDEXnectaroutsideArray[t]

In [ ]:
def INDEXpollenoutside(t) :
    # (25)
    if t > len(INDEXpollenoutsideArray):
        INDEXpollenoutside(t-1)
    elif t == len(INDEXpollenoutsideArray):
        res = min((1 - season(t)) * 1.5, 1)
        INDEXpollenoutsideArray = np.append(INDEXpollenoutsideArray, res)
    return INDEXpollenoutsideArray[t]

## 2.5 Modeling task decisions

In [ ]:
def NEEDworkers(t):
    # (26)
    return NEEDnurses(t) + NEEDpollenforagers(t)

In [ ]:
def RATIOworkforce(t, FACTORothertasks = 0.2):
    # (27)
    return min(BEESadult(t) * (1 - FACTORothertasks) / (NEEDWORKERS(t) + 1), 1)

## 2.6 Modeling the regulation of nursing

In [ ]:
def NEEDnurses(t):
    # (28)
    res = 0
    for i in range(1, LIFESPANlarva):
        res = LARVAE(i, t) * NEEDnurses_per_larva(i)
    return res + CELLSeggs(t,opt) * NEEDnursesperegg + CELLSpupae(t,opt3) * NEEDnursesperpupa

In [ ]:
def NURSES(t):
    # (29)
    return NEEDworkers(t) + RATIOworkforce(t)

In [ ]:
def INDEXnursingquality(t):
    # (30)
    return NURSES(t)/(NEEDnurses(t) + 1)

## 2.7 Modeling the regulation of foraging

In [ ]:
def FORAGERS(t):
    # (31) represents the available workforce for the foraging task.
    res = FORAGERSpollen(t)+FORAGERSnectaractive(t)
    return(res)

In [ ]:
def FORAGERSactive(t):
    # (32) represents the actual number of foragers that fly out
    res = FORAGERSpollenactive(t)+FORAGERSnectaractive(t)
    return(res)

### 2.7.1 Recruitment of pollen foragers

In [ ]:
def NEEDpollen(t):
    # (33) calculate the colony’s pollen demand
    res = NEEDpollen_larvae(t)+NEEDpollen_adult(t)
    return(res)

In [ ]:
def NEEDpollen_larvae(t):
    # (34) represents the pollen demand of larvae of all ages
    result = 0
    for i in range(1,LIFESPANlarva):
        result+= (POLLENNEEDlarva[i-1]*(LARVAE(i,t)))
    return(result)

In [ ]:
def NEEDpollen_adult(t):
    # (35) represents the adult’s pollen demand
    result = BEESadult(t) * POLLENNEEDadult + NURSES(t) * POLLENNEEDnurse
    return(result)

In [ ]:
def NEEDpollenincome(t):
    # (36) represents the daily need for pollen income
    S = 0
    for i in range(0,2):
        S += NEEDpollen(t - i)
    tmp = S/3 * FACTORpollenstorage - STORESpollen(t)
    result = max(0,tmp)
    return(result)

In [ ]:
def NEEDpollenforagers(t):
    # (37) models the number of pollen foragers needed according to the current need for additional pollen.
    result = NEEDpollenincome(t-1) / (LOADpollenforager * TURNSpollenforager * FACTORforagingsuccess)
    return(result)

In [ ]:
def FORAGERSpollen(t):
    # (38) models the potential number of pollen foragers each day
    max1 = NEEDpollenforagers(t) * RATIOworkforce(t)
    max2 = (BEESadult(t)-NURSES(t))*FACTORminpollenforagers
    min1 = max(max1,max2)
    min2 = BEESadult(t) * FACTORforagingmax
    result = min(min1,min2)
    return(result)

In [ ]:
def FORAGERSpollenactive(t):
    # (39) models the number of foragers that actually leave the hive for foraging flights
    result = FORAGERSpollen(t) * INDEXflight(t) * INDEXpollenoutside(t)
    return(result)




### 2.7.2 Recruitment of nectar foragers

In [ ]:
def NEEDnectar(t):
    # (40) model the demand for nectar
    result = NEEDnectar_larvae(t) + NEEDnectar_adult(t)
    return(result)

In [ ]:
def NEEDnectar_larvae(t):
    # (41) model the demand for larvae nectar
    result = 0
    for i in range(1,LIFESPANlarva):
        result += NECTARNEEDlarva[i-1] * LARVAE(i,t)
    return(result)

In [ ]:
def NEEDnectar_adult(t):
    # (42)model the demand for adult nectar
    result = BEESadult(t) * NECTARNEEDadult + NURSES(t) * NECTARNEEDnurse + FORAGESactive(t) * NECTARNEEDactiveforager
    return(result)

In [ ]:
def WORKFORCEnectar(t):
    # (43) calculates the number of adult bees that are not involved in other tasks and thus are still available for a task associated with nectar collection
    result = 0
    if (RATIOworkforce(t) == 1):
        result = (BEESadult(t) * (1 - FACTORothertasks)) - NURSES(t) - FORAGERSpollen(t)
    return(result)

In [ ]:
def FORAGERSnectar(t):
    # (44) models the number of potential nectar foragers
    min1 = (BEESadult(t) * FACTORforagingmax) - FORAGERSpollen(t)
    min2 = WORKFORCEnectar(t) - PROCESSORS(t)
    result = min(min1,min2)
    return(result)

In [ ]:
def FORAGERSnectaractive(t): 
    # (45) models the number of nectar foragers that actually leave the hive for foraging flights
    result = FORAGERSnectar(t) * INDEXflight(t) * INDEXnectaroutside(t)
    return(result)

## 2.8 Modeling the resource influx into the colony

In [ ]:
def INCOMEpollen(t):
    # (46) project the daily influx of pollen
    result = FORAGERSpollenactive(t) * LOADpollenforager * TURNSpollenforager * FACTORforagingstoch(t) * FACTORforagingsuccess
    return(result)

In [ ]:
def FACTORforagingstoch(t):
    # (47) used to vary the daily foraging success symmetrically around 1
    if (stochastic_factor > 0.24 and stochastic_factor < 0.26):
        result = random.randint(75,125) /100
    else:
        result = 1
    return(result)

In [ ]:
def INDEXpollensituation(t):
    # (48) describes the level of the pollen stores in relation to the demand situation of the colony
    min1 = STORESpollen(t) / (NEEDpollen(t) * FACTORpollenstorage + 1)
    result = min(1,min1)
    return(result)

In [ ]:
def INCOMEnectar(t):
    # (49) project the daily influx of nectar
    min1 = FORAGERSnectaractive(t) * LOADnectarforager * TURNSnectarforager * FACTORforagingstoch(t) * FACTORforagingsuccess
    min2 = CELLSempty(t - 1)
    result = min(min1,min2)
    return(result)

## 2.9 Regulation of food processing

In [ ]:
#Needed nectar processors on day t
def NEEDprocessors(t):
    #(50)
    result = STORESnectar(t-1)*ProcessorsPerCell
    return(result)

In [ ]:
def PROCESSORS(t):
    #(51)
    result = min(NEEDprocessors(t), WORKFOREnectar(t))
    return(result)

In [ ]:
def PROCESSEDnectar(t):
    #(52)
    result = min(STORESnectar(t-1)-USAGEnectar(t), PROCESSORS(t)/ProcessorsPerCell)
    return(result)

## 2.10 Management of nutrient stores

In [ ]:
#The daily usage (consumption) of pollen
def USAGEpollen(t):
    #(53)
    result = min(dFResult.loc[dFResult.index[t-1],'STORESpollen'], NEEDpollen(t-1)*(1-(FACTORpollensavingmax*(1-INDEXpollensituation(t-1)))))
    return(result)

In [ ]:
#The daily amount of stored pollen
def STORESpollen(t):
    #(54)
    if t<=0:
        t=0
    result = dFResult.loc[dFResult.index[t-1],'STORESpollen'] + INCOMEpollen(t) - USAGEpollen(t)
    return(result)

In [ ]:
def USAGEnectar(t):
    #(55)
    result = min(dFResult.loc[dFResult.index[t-1],'STORESnectar'], NEEDnectar(t))
    return(result)

In [ ]:
def USAGEhoney(t):
    #(56)
    result = min(dFResult.loc[dFResult.index[t-1],'STOREShoney'], (NEEDnectar(t)-USAGEnectar(t))*RATIOnectar_to_honey)
    return(result)

In [ ]:
def STORESnectar(t):
    #(57)
    result = dFResult.loc[dFResult.index[t-1],'STORESnectar']+INCOMEnectar(t) - USAGEnectar(t)-PROCESSEDnectar(t)
    return(result)

In [ ]:
def STOREShoney(t):
    #(58)
    result = dFResult.loc[dFResult.index[t-1],'STOREShoney']-USAGEhoney(t)+(PROCESSEDnectar(t)*RATIOnectar_to_honey)
    return(result)

In [ ]:
def WEIGHTcolony(t):
    #(59)
    res = 0
    for i in range(1, LIFESPANlarva):
        res = res + (w_larva[i]*LARVAE(i,t))
    
    result = 1/1000*(w_hivebase+w_cellsbase*CELLShive+w_pollen*STORESpollen(t)
            +w_honey*STOREShoney(t)+w_egg*CELLSeggs(t,opt)
            +w_pupa*CELLSpupae(t,opt3)
            +res
            +w_adult*BEESadult(t))
    return(result)

In [ ]:
def BEESlazy(t):
    #(60)
    result = (BEESadult(t)*(1-FACTORothertasks))-FORAGERSactive(t)-NURSES(t)-PROCESSORS(t)
    return(result)

In [ ]:
def FACTORqueen(t):
  # (61)
    if ((swd - 30) <= t and t < (swd -3)):
        res = 1.2
    elif ((swd - 3) <= t and t < (swd + 14)):
        res = 0
    else:
        res = 1
    return res

In [ ]:
def ELRbase(t):
  # (62)
    if (t<swd):
        res = 2000
    else:
        res = 1200
    return res

In [ ]:
def BEESswarm(t):
  # (63)
    if (t == swd):
        res = BEESadult(t-1) * 0.6
    else:
        res = 0
    return res

In [ ]:
def TURNSnectarforager(t):
    # (64)
    return 15 + 7 * ((CELLSempty(t-1))/(CELLShive+1))

In [ ]:
def LOSShoney_swarm(t):
    # (65)
    if t == swd:
        return min(dFResult.loc[dFResult.index[t-1],'STOREShoney'],BEESswarm(t)*LOADnectarforager)
    else:
        return 0

In [ ]:
def USAGEhoney(t):
    # (56a)
    return min(dFResult.loc[dFResult.index[t-1],'STOREShoney'],((NEEDnectar(t)-USAGEnectar(t))*RATIOnectar_to_honey)+LOSShoney_swarm(t))

In [ ]:
def CELLShive(t):
    # (67)
    if t <= swd:
        return 0
    elif swd < t and t <= (swd + 7):
        return CELLShive(t-1)+(2000/7)
    elif (swd + 7) < t and t <= (swd + 97):
        return CELLShive(t-1)+(8000/90)
    elif t > (swd + 97):
        return CELLShive(t-1)

In [ ]:
for i in range(1,365) :
    dFResult.loc[dFResult.index[i],'STOREShoney'] = STOREShoney(i)
    dFResult.loc[dFResult.index[i],'STORESnectar'] = STORESnectar(i)
    dFResult.loc[dFResult.index[i],'STORESpollen'] = STORESpollen(i)
    dFResult.loc[dFResult.index[i],'BroodCells'] = CELLSbrood(i)
    dFResult.loc[dFResult.index[i],'BEESadult'] = BEESadult(i)
    dFResult.loc[dFResult.index[i],'WEIGHTcolony'] = WEIGHTcolony(i)


In [ ]:
print(dFResult.loc[dFResult.index[1],'STOREShoney'])

In [ ]:
print(STOREShoney(1))